In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# In order for GPU to work, we must run this code
import shutil
import re

original_path = '/usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py'
with open(original_path, 'r') as file:
  content = file.read()
  content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
  content = re.sub(r'control_flow_ops.case', 'tf.case', content)
  content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
  with open(original_path, 'w') as file:
    file.write(content)

print(f"File {original_path} fixed.")

File /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py fixed.


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git
# navigate to /models/research folder to compile protos
%cd models/research
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# test the model builder
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# run this to go back to home directory
%cd ..
%cd ..

/content/models
/content


In [ ]:
# We are using the model SSD MobileNet v2 640x640 this downloads and extracts it to root directory, for ease of access
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

!tar -xzvf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

In [ ]:
# unzip the training data, if using large roboflow dataset
!unzip "/content/drive/MyDrive/Models/pretraining-data.v2i.tfrecord.zip"

unzip:  cannot find or open /content/drive/MyDrive/Models/pretraining-data.v2i.tfrecord.zip, /content/drive/MyDrive/Models/pretraining-data.v2i.tfrecord.zip.zip or /content/drive/MyDrive/Models/pretraining-data.v2i.tfrecord.zip.ZIP.


# Train the Model

In [ ]:
# Navigate to object_detection
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
import os
num_cpu_cores = os.cpu_count()
print(num_cpu_cores)

8


In [ ]:
with tf.device('/GPU:0'):
  print("Is GPU available:", tf.test.is_gpu_available())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Is GPU available: True


In [ ]:
# THIS IS THE CODE TO START TRAINING

with tf.device('/GPU:0'):
  !python model_main_tf2.py\
    --pipeline_config_path=/content/drive/MyDrive/Models/golf-ssd-640/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config\
    --model_dir=/content/drive/MyDrive/Models/golf-ssd-640/training/model-checkpoint-storage\
    --alsologtostderr

2024-01-28 00:52:56.251149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 00:52:56.251199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 00:52:56.252393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 00:52:57.197897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-28 00:52:59.687758: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using Mir

In [ ]:
# Evaluate the model:
!python model_main_tf2.py\
  --pipeline_config_path=/content/drive/MyDrive/Models/golf-ssd-640/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config\
  --model_dir=/content/drive/MyDrive/Models/golf-ssd-640/training\
  --checkpoint_dir=/content/drive/MyDrive/Models/golf-ssd-640/training/training-checkpoints\
  --num_workers=8\
  --sample_1_of_n_eval_examples=1

In [ ]:
# export the model:
!python exporter_main_v2.py \
    --trained_checkpoint_dir=/content/drive/MyDrive/Models/golf-ssd-640/training/model-checkpoint-storage \
    --pipeline_config_path=/content/drive/MyDrive/Models/golf-ssd-640/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config \
    --output_directory /content/drive/MyDrive/Models/golf-ssd-640/inference/model-3-28k-steps

2024-01-28 01:06:58.261376: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 01:06:58.261422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 01:06:58.262660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 01:06:59.232770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-28 01:07:01.677581: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating